# eSafety Hackathon - Data Exploration

This notebook provides a starting point for exploring the hackathon data.

## Setup

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import sys
sys.path.append('..')

# Our utilities
from src.utils.text_utils import clean_text, count_features, detect_patterns, extract_emojis

# Display settings
pd.set_option('display.max_colwidth', 200)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete! ✅")

## Load Data

Update the path below to point to your hackathon data.

In [ ]:
# TODO: Update this path to your data file
DATA_PATH = '../data/raw/messages.csv'  # or .json, .parquet, etc.

# Example loading (uncomment the one that matches your data format)
# df = pd.read_csv(DATA_PATH)
# df = pd.read_json(DATA_PATH)
# df = pd.read_parquet(DATA_PATH)

# For now, create sample data to test the pipeline
sample_data = [
    {"id": 1, "user": "user_001", "text": "I just landed my DREAM job after being rejected 47 times. Never give up! Agree? 🚀"},
    {"id": 2, "user": "user_002", "text": "I wish I could look like her. Skipped lunch again today."},
    {"id": 3, "user": "user_003", "text": "Had a great coffee this morning! ☕"},
    {"id": 4, "user": "user_004", "text": "Nobody understands. Society is rigged against people like us."},
    {"id": 5, "user": "user_001", "text": "I'm HUMBLED to announce I've been selected as a thought leader in disruption. 🙏"},
]

df = pd.DataFrame(sample_data)
print(f"Loaded {len(df)} messages")
df.head()

## Basic Statistics

In [ ]:
# Basic stats
print(f"Total messages: {len(df):,}")
print(f"Unique users: {df['user'].nunique():,}")
print(f"\nMessages per user:")
print(df['user'].value_counts().describe())

In [ ]:
# Extract text features
df['features'] = df['text'].apply(count_features)

# Expand features into columns
features_df = pd.json_normalize(df['features'])
df = pd.concat([df, features_df], axis=1)

# Show feature distributions
print("Text Feature Statistics:")
df[['word_count', 'emoji_count', 'caps_ratio', 'exclamation_count']].describe()

## Pattern Detection

In [ ]:
# Detect patterns
df['patterns'] = df['text'].apply(detect_patterns)

# Show which patterns were detected
for idx, row in df.iterrows():
    detected = [k for k, v in row['patterns'].items() if v]
    if detected:
        print(f"Message {row['id']}: {detected}")
        print(f"  Text: {row['text'][:100]}...\n")

## User-Level Analysis

For persona classification, we might want to aggregate at the user level.

In [ ]:
# Aggregate by user
user_stats = df.groupby('user').agg({
    'text': 'count',
    'word_count': 'mean',
    'emoji_count': 'sum',
    'exclamation_count': 'sum',
    'caps_ratio': 'mean'
}).rename(columns={'text': 'message_count'})

user_stats

## Visualization

In [ ]:
# Word count distribution
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].hist(df['word_count'], bins=30, edgecolor='black')
axes[0, 0].set_title('Word Count Distribution')
axes[0, 0].set_xlabel('Words per message')

axes[0, 1].hist(df['emoji_count'], bins=20, edgecolor='black')
axes[0, 1].set_title('Emoji Count Distribution')
axes[0, 1].set_xlabel('Emojis per message')

axes[1, 0].hist(df['caps_ratio'], bins=20, edgecolor='black')
axes[1, 0].set_title('Caps Ratio Distribution')
axes[1, 0].set_xlabel('Ratio of uppercase letters')

# Messages per user
df['user'].value_counts().head(20).plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Top 20 Users by Message Count')
axes[1, 1].set_xlabel('User')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Most Common Words/Phrases

In [ ]:
from collections import Counter
import re

# Get all words
all_text = ' '.join(df['text'].apply(lambda x: clean_text(x, remove_punctuation=True)))
words = all_text.split()

# Filter out common stop words
stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
              'to', 'of', 'and', 'in', 'that', 'it', 'for', 'on', 'with', 'as', 'at',
              'by', 'this', 'from', 'or', 'but', 'not', 'you', 'your', 'i', 'my', 'me',
              'we', 'our', 'they', 'their', 'he', 'she', 'his', 'her', 'have', 'has'}

filtered_words = [w for w in words if w not in stop_words and len(w) > 2]

print("Top 30 words:")
for word, count in Counter(filtered_words).most_common(30):
    print(f"  {word}: {count}")

## Next Steps

1. **Run the classification notebook** (`02_classification.ipynb`) to test different approaches
2. **Explore user patterns** - Do certain users consistently post in specific styles?
3. **Look for clusters** - Use embeddings to group similar messages
4. **Check for temporal patterns** - If timestamps available, look for posting patterns